In [1]:
#Goal is to create a neural network evaluate function. For now, this is simply a preditor of the which side will win

In [292]:
import chess
import pandas as pd
games = pd.read_csv('Chess DS\games.csv')

In [293]:
def game_to_pos(game): 
    # function that takes a Series (given as a list of SAN whitespace separated moves) and outputs a list of pairs
    board = chess.Board()
    serial = []
    winner_dict = {'white':1, 'draw':0, 'black':-1}
    result = winner_dict[game['winner']]
    after_ply = 0 
    for move in game['moves'].split():
        serial.append((board.copy(), result,after_ply)) 
        board.push_san(move)
        after_ply += 1
        
    return pd.DataFrame(serial, columns =  ['board', 'result', 'ply'])

In [294]:
%%time
mydataset = pd.concat([game_to_pos(row) for idx, row in games[:4000].iterrows()])

Wall time: 1min 16s


In [295]:
len(mydataset)

236207

In [296]:
import tensorflow as tf
import sklearn.model_selection
import numpy as np

In [297]:
mydataset.reset_index(drop = True, inplace = True)
ply_filtered_set = mydataset[mydataset['ply']>15]
train, test = sklearn.model_selection.train_test_split(ply_filtered_set, train_size = .9)

In [298]:
len(train)

156371

In [299]:
#take board to 772 one-hot encoding
def board_to_vector(board):
    piece_types = [chess.KING, chess.QUEEN, chess.ROOK, chess.BISHOP, chess.KNIGHT, chess.PAWN]
    colors  = [chess.WHITE, chess.BLACK]
    fullvect = np.array([])
    for piece_type in piece_types: 
        for color in colors: 
            a = np.zeros(64)
            a[list(board.pieces(piece_type, color))] = 1
            fullvect = np.append(fullvect,a)
    castling =  np.array([bool(board.castling_rights & chess.BB_H1)*1, bool(board.castling_rights & chess.BB_H8)*1,\
                      bool(board.castling_rights & chess.BB_A1)*1, bool(board.castling_rights & chess.BB_A8)*1])
    return np.append(fullvect, castling)

In [300]:
model = tf.keras.Sequential([tf.keras.Input((772,)), tf.keras.layers.Dense(100), tf.keras.layers.Dense(100), \
                             tf.keras.layers.Dense(1)])
model.summary()
model.compile(loss = 'mean_squared_error', optimizer = 'adam')

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 100)               77300     
_________________________________________________________________
dense_11 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 101       
Total params: 87,501
Trainable params: 87,501
Non-trainable params: 0
_________________________________________________________________


In [301]:
%%time
X = np.vstack(train['board'].apply(board_to_vector))
Y = train['result'].to_numpy()

Wall time: 43.1 s


In [303]:
%%time
model.fit(x = X, y = Y, epochs =5)

Epoch 1/5
4887/4887 [==============================] - 8s 2ms/step - loss: 0.7053
Epoch 2/5
4887/4887 [==============================] - 9s 2ms/step - loss: 0.7046
Epoch 3/5
4887/4887 [==============================] - 8s 2ms/step - loss: 0.7037
Epoch 4/5
4887/4887 [==============================] - 8s 2ms/step - loss: 0.7031
Epoch 5/5
4887/4887 [==============================] - 8s 2ms/step - loss: 0.7024
Wall time: 42.7 s


In [315]:
%%time
model.predict(np.array(\
    [ board_to_vector(chess.Board(fen = 'rn1qkbnr/pppbpppp/8/3P4/2P5/8/PP1P1PPP/RNBQKBNR w KQkq - 0 1'))]))[0][0]


Wall time: 79.8 ms


0.13249034

In [305]:
mydataset.to_pickle('mydataset')

In [307]:
np.save('one_hot_train', X)

In [309]:
model.save('model1')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model1\assets


In [316]:
9

9